In [108]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [109]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True
    
    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    

    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        # self.output = input 
        # return self.output
        
        raise NotImplementedError()

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        raise NotImplementedError()
    
    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [110]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
            
        Just write a little loop. 
        """

        # Your code goes here. ################################################
        input = np.copy(input)
        for i, m in enumerate(self.modules):
            input = m.forward(input)
        self.output = input
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        !!!
                
        To each module you need to provide the input, module saw while forward pass, 
        it is used while computing gradients. 
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass) 
        and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        # Your code goes here. ################################################
        
        g = np.copy(gradOutput)
        for i, (m, m_prev) in enumerate(zip(reversed(self.modules), reversed(self.modules[:-1]))):
            # don't need to .accGradParameters()
            # as it's called by .backward()
            g = m.backward(m_prev.output, g)
        self.gradInput = self.modules[0].backward(input, g)
        return self.gradInput

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)
    
    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()
    
    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1. Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

* * *

### Derivation
I kindly ask to excuse me, but it's a period in my life when
I have to treat $\mathbb{R}^n$ as a _manifold_
and backprop as a _pullback_.

$$x \in \operatorname{Mat}(\mathrm{sampleSize}, \mathrm{nInputFeatures}),$$
$$W \in \operatorname{Mat}(\mathrm{nOutFeatures}, \mathrm{nInFeatures}),$$
$$y = F(x) = X W^\top + b\in\operatorname{Mat}(\mathrm{sampleSize}, \mathrm{nOutFeatures}).$$

Having a mapping $$\operatorname{Mat}(\mathrm{nOutFeatures}, \mathrm{nInFeatures})\to\operatorname{Mat}(\mathrm{sampleSize}, \mathrm{nOutFeatures})$$
we want to pullback a cotangent
$$\left(\operatorname{Mat}(\mathrm{sampleSize}, \mathrm{nOutFeatures})\right)^*$$
to $$(\operatorname{Mat}(\mathrm{nOutFeatures}, \mathrm{nInFeatures}))^*$$
and convert it into a tangent via inverse metric tensor (which in our case is an identity).

Though it's easier to work with a single observation, throwing away sample dimension (we'll have just a sum over sample dim anyway).

* * *
So we just need to write a row-centric version of usual pushforwards and pullbacks.
So, assuming $\mathrm{simpleSize}=1$:
$$F_{x,*}X = XW^\top,$$
$$(F_x^*\eta) = \eta(F_{x,*}X) = \eta(XW^\top) = XW^\top \eta = \operatorname{Trace}\langle \eta X, W\rangle ,$$
$$(F_x^*\eta)^\sharp = \eta X$$

Now with $X$ of shape `(n_sample, n_in)`, $Y = XW^\top + \operatorname{broadcast}(b)$ of shape `(n_sample, n_out)`

The right cotangent is $\eta : \operatorname{Mat}(\mathrm{n\_sample}, \mathrm{n\_out})\to\mathbb{R}$,
with action easier to be described as $\eta(A) = \operatorname{Trace}\langle \eta^\sharp, A\rangle.$

Pullback cotangent then is $\xi: \operatorname{Mat}(\mathrm{n\_sample}, \mathrm{n\_in})\to\mathbb{R}$,
$$
\xi(H) = \eta(XH^\top) = \operatorname{Trace}\langle \eta^\sharp, XH^\top \rangle
= \operatorname{Trace}\langle \eta^{\sharp\top} X, H\rangle = \sum_{ijk}\eta^\sharp_{ij} X_{ik}H_{jk}$$

## TLDR

Changed notation a few times yesterday, here's final version,
involving placeholder ($[\cdot]$) notation for anonymous functions,
denoting right derivative with $\eta$ and left with $\xi$,
and corresponding gradients with sharps ($\sharp$):

$$W\mapsto Y = XW^\top + b.$$
Pullback takes
$$\eta = \langle [\cdot], \eta^\sharp \rangle
\mapsto \xi = \langle [\cdot], \xi^\sharp \rangle
= \operatorname{Trace}([\cdot]^\top \xi^\sharp)
= \langle X [\cdot]^\top, \eta^\sharp\rangle
= \operatorname{Trace}([\cdot] X^\top \eta^\sharp)$$

So the pulled-back _gradient_ is $$\xi^\sharp = \eta^{\sharp\top}X$$

In [111]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, InnerProductLayer in caffe. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        # self.output = ...
        #self.output = np.matmul(input, np.transpose(self.W), out=self.output)
        # TODO: out=subarray, for the last batch
        self.output = np.matmul(input, np.transpose(self.W))
        np.add(self.output, self.b.reshape((1, -1)), out=self.output)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        # self.gradInput = ...
        self.gradInput = np.matmul(gradOutput, self.W)
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        # self.gradW = ... ; self.gradb = ...
        np.add(self.gradb, gradOutput.sum(axis=0), out=self.gradb)
        np.add(self.gradW,
               np.matmul(np.transpose(gradOutput), input),
               out=self.gradW)
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## 2. SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

* * *

Again, assuming inconvenient row-centric point of view:
for a $F:M{=}{^m}\mathbb{R}\to N{=}{^n}\mathbb{R}$
right tangents are rows ${^n}\mathbb{R}$,
right cotangents are $\mathbb{R}^n$,
pushforwards act as $Y = F_*X = XJ(F) \in {^n}\mathbb{R}$,
pullbacks as $\eta = F^*\xi = J(F)\eta \in \mathbb{R}^m$,
with $J(F) = (\partial_i F_j)$ (note the **inversion of indices**),
and finally **for `bs=1`**
$$S(x) = \exp(x)\frac{1}{\exp(x)\mathbf{1}^\top},$$
$$\partial_j s_i(x) = \delta_{ij} s_i(x) - s_i(x)s_j(x).$$

$$x\mapsto \operatorname{softmax}(x),$$
$$X \mapsto XJ(\operatorname{softmax}),$$
$$J = (J_{ij} = \partial_i s_j = \delta_{ij} s_j - s_j s_i) = \operatorname{diag}(s) - s^\top\otimes s,$$

$$\eta = \langle [\cdot], \eta^\sharp\rangle \mapsto \xi = \langle[\cdot], \xi^\sharp\rangle
= \langle [\cdot]J, \eta^\sharp\rangle,$$
$$\xi^\sharp = \eta^\sharp J = \eta^\sharp (\operatorname{diag}(s) - s^\top \otimes s) = \eta^\sharp\odot s - \eta^\sharp s^\top s,$$

In [112]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    @staticmethod
    def _softmax(input, output, subtract=True):
        if output is None or output.shape != input.shape:
            output = np.empty_like(input)
        if subtract:
            output = np.subtract(input, input.max(axis=1, keepdims=True), out=output)
        assert np is not None
        np.exp(output, out=output)
        np.divide(output, output.sum(axis=1, keepdims=True), out=output)
        return output
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # Your code goes here. ################################################
        self.output = self._softmax(input, output=self.output, subtract=False)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        bs = input.shape[0]
        s = self._softmax(input, output=None, subtract=True)
        masked = np.multiply(s, gradOutput)
        self.gradInput = np.zeros_like(input)
        for i in range(bs):
            self.gradInput[i] = masked[i] - masked[i].sum() * s[i]
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

## 3. LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

$$J_{ij} = \delta_{ij} - \frac{1}{\exp x \mathbf{1}^\top}\exp x_i = \delta_{ij} - s_j,$$

$$\xi^\sharp = \eta^\sharp J = \eta^\sharp - \eta^\sharp \mathbf{1}^\top s.$$

In [113]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()
    
    @staticmethod
    def __logsoftmax(input, output, subtract=True):
        if subtract:
            np.subtract(input, input.max(axis=1, keepdims=True), out=output)
        tmpsum = np.log(np.exp(output).sum(axis=1, keepdims=True))
        np.subtract(output, tmpsum, out=output)
        return output
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # Your code goes here. ################################################
        self.output = self.__logsoftmax(input,
                                        output=self.output,
                                        subtract=False)
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        # xi <- s
        self.gradInput = SoftMax._softmax(input, output=self.gradInput)
        
        # xi <- s*sum(eta)
        np.multiply(self.gradInput,
                    np.expand_dims(gradOutput.sum(axis=-1), axis=-1),
                    out=self.gradInput)
        # xi <- -s*sum(eta)
        np.multiply(self.gradInput, -1, out=self.gradInput)
        # xi <- eta - s*sum(eta)
        np.add(gradOutput, self.gradInput, out=self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "LogSoftMax"

## 4. Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance: 
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance. 

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [114]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None
    
    def updateMovingAverages(self, input):
        mean = np.mean(input, axis=0)
        var = np.var(input, axis=0)
        if self.moving_mean is None:
            self.moving_mean = mean
            self.moving_variance = var
        else:
            for cur, mov in [(mean, self.moving_mean), (var, self.moving_variance)]:
                np.multiply(mov, self.alpha, out=mov)
                np.multiply(cur, 1. - self.alpha, out=cur)
                np.add(mov, cur, out=mov)
    
    def _invstd(self, var):
        var = np.copy(var)
        var = np.add(var, self.EPS, out=var)
        std = np.sqrt(var, out=var)
        invstd = np.divide(1, std, out=var)
        return invstd
        
    
    def updateOutput(self, input):
        # Your code goes here. ################################################
        # use self.EPS please
        if self.training:
            # bullshit, I'd rather use moving averages
            # during training as well
            # -- both cleaner code and more meaningful model
            self.updateMovingAverages(input)
            mean = np.mean(input, axis=0, keepdims=True)
            var = np.var(input, axis=0, keepdims=True)
        else:
            mean = np.expand_dims(self.moving_mean, 0) # .freeze() that doesn't exist in numpy
            var = np.expand_dims(self.moving_variance, 0) # .freeze(), CoW is nonexistent as well, afaik
            mean, var = np.copy(mean), np.copy(var) # just to make sure
        self.output = np.subtract(input, mean, out=self.output)
        invstd = self._invstd(var)
        self.output = np.multiply(self.output, invstd, out=self.output)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        bs = input.shape[0]
        QX = input - np.mean(input, axis=0, keepdims=True)
        invstd = self._invstd(np.var(input, axis=0, keepdims=True))
        Y = np.multiply(QX, invstd)
        gsum = np.sum(gradOutput, axis=0)
        dotp = np.sum(np.multiply(QX, gradOutput), axis=0)
        k = dotp*invstd*invstd/bs
        self.gradInput = QX*k
        self.gradInput = gradOutput - gsum/bs - self.gradInput
        self.gradInput = self.gradInput*invstd
        return self.gradInput
    
    def __repr__(self):
        return "BatchNormalization"

The original paper is such a mess that I'm not even going to try to write down underlying math properly -- not worth the time

In [115]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)
        
        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output
        
    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)
    
    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)
        
    def getParameters(self):
        return [self.gamma, self.beta]
    
    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]
    
    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [116]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        if self.training:
            self.mask = np.random.binomial(1, 1. - self.p, size=input.shape).astype(float)
            np.divide(self.mask, 1.-self.p, out=self.mask)
            # won't work because unittests change batchsize
            # self.output = np.multiply(self.mask, input, out=self.output)
            self.output = np.multiply(self.mask, input)
        elif self.output is None:
            self.output = np.copy(input)
        else:
            np.copyto(dst=self.output, src=input)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = self.mask*gradOutput
        return self.gradInput
        
    def __repr__(self):
        return "Dropout"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**): 

In [117]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

## 6. Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope. 

In [118]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.multiply(input, input > 0) + self.slope * np.multiply(input, input < 0) # don't care
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.multiply(gradOutput, input > 0) + self.slope * np.multiply(gradOutput, (input < 0).astype(float))
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

## 7. ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [119]:
class ELU(Module):
    def __init__(self, alpha=1.0):
        super(ELU, self).__init__()
        
        self.alpha = alpha
    
    @staticmethod
    def _elu(input, output, alpha):
        where = input < 0
        if output is None or output.shape != input.shape:
            output = np.empty_like(input)
        np.copyto(dst=output, src=input)
        np.exp(output, out=output, where=where)
        np.add(output, -1, out=output, where=where)
        np.multiply(output, alpha, out=output, where=where)
        return output
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = self._elu(input, self.output, self.alpha)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if self.gradInput is None or self.gradInput.shape != input.shape:
            self.gradInput = np.ones_like(input)
        else:
            self.gradInput.fill(1)
        where = input <= 0
        np.copyto(dst=self.gradInput, src=input, where=where)
        np.exp(self.gradInput, out=self.gradInput, where=where)
        np.multiply(self.gradInput, self.alpha, out=self.gradInput, where=where)
        np.multiply(self.gradInput, gradOutput, out=self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "ELU"

## 8. SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [120]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()
    
    def updateOutput(self, input):
        # Your code goes here. ################################################
        if self.output is None or self.output.shape != input.shape:
            self.output = np.empty_like(input)
        self.output = np.exp(input, out=self.output)
        np.add(self.output, 1, out=self.output)
        np.log(self.output, out=self.output)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if self.gradInput is None or self.gradInput.shape != input.shape:
            self.gradInput = np.empty_like(input)
        self.gradInput = np.multiply(input, -1, out=self.gradInput)
        np.exp(self.gradInput, out=self.gradInput)
        np.add(self.gradInput, 1, out=self.gradInput)
        np.divide(1., self.gradInput, out=self.gradInput)
        np.multiply(self.gradInput, gradOutput, out=self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "SoftPlus"

# Criterions

Criterions are used to score the models answers. 

In [121]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [122]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 9. Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula, 
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [123]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()
        
    def updateOutput(self, input, target): 
        
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        
        # Your code goes here. ################################################
        bs = input.shape[0]
        input_clamp = np.log(input_clamp, out=input_clamp)
        np.multiply(input_clamp, target, out=input_clamp)
        self.output = -1./bs * np.sum(input_clamp)
        return self.output

    def updateGradInput(self, input, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
                
        # Your code goes here. ################################################
        if self.gradInput is None:
            self.gradInput = np.empty_like(input)
        np.divide(target, input_clamp, out=self.gradInput)
        np.divide(self.gradInput, -input.shape[0], out=self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 10. Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [124]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()
        self._output = None
        
    def updateOutput(self, input, target): 
        # Your code goes here. ################################################
        if self._output is None or self._output.shape != input.shape:
            self._output = np.empty_like(input)
        self._output = np.multiply(input, target, out=self._output)
        self.output = -1./input.shape[0] * np.sum(self._output)
        return self.output

    def updateGradInput(self, input, target):
        # Your code goes here. ################################################
        self.gradInput = -1./input.shape[0] * target
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterion"

# 11. Contrastive criterion

- input:   **`batch_size x n_feats`** - embedding vectors (can be outputs of any layer, but usually the last one before the classification linear layer is used)
- target: **`labels`** - ground truth class indices (not one-hot ecodings)
- output: **scalar**


The contrastive loss pulls the examples of the same class closer (in terms of the embedding layer) and pushes the examples of different classes from each other.

This is the formula for the contrastive loss in the terms of embedding pairs:
$$L_c(x_i, x_j, label_i, label_j) = \begin{cases}\frac{1}{N_{pos}} \| x_i - x_j \|^{2}_{2} & label_i = label_j\\ \frac{1}{N_{neg}} (\max(0, M - \| x_i - x_j \|_{2}))^2 & label_i \neq label_j \end{cases}$$
Where $M$ is a hyperparameter (constant), 
$N_{pos}$ and $N_{neg}$ - number of 'positive' (examples of the same class) and 'negative' pairs (examples of different classes).

You should generate all the possible pairs in the batch (ensure that your batch size is > 10, so that there are positive pairs), but remember to be efficient and use vectorize opetations, hint: https://medium.com/dataholiks-distillery/l2-distance-matrix-vectorization-trick-26aa3247ac6c. Then compute the Euclidean distances for them, and finally compute the loss value.

When computing the gradients with respect to inputs, you should (as always) use the chain rule: compute the loss gradients w.r.t distances, compute the distance gradients w.r.t input features (each example may take part in different pairs).



In [125]:
class ClassContrastiveCriterion(Criterion):
    def __init__(self, M):
        a = super(ClassContrastiveCriterion, self)
        super(ClassContrastiveCriterion, self).__init__()
        self.M = M #margin for punishing the negative pairs
        
    @staticmethod
    def compute_distances_no_loops(X):
        """
`Source <https://medium.com/dataholiks-distillery/l2-distance-matrix-vectorization-trick-26aa3247ac6c`_"""
        dists = (
            -2 * np.dot(X, X.T)
            + np.sum(X**2, axis=1)
            + np.sum(X**2, axis=1)[:, np.newaxis]
        )
        np.sqrt(dists, out=dists)
        return dists
        
    def updateOutput(self, input, target): 
        # Your code goes here. ################################################
        neg = np.expand_dims(target, axis=1)
        neg = (neg - np.transpose(neg)) != 0
        nneg = np.sum(neg)
        npos = neg.size - nneg
        self.output = self.compute_distances_no_loops(input)
        np.add(-self.M, self.output, where=neg,
               out=self.output)
        np.minimum(self.output, 0, where=neg,
                   out=self.output)
        np.power(self.output, 2,
                 out=self.output)
        np.multiply(self.output, 1./nneg, where=neg,
                    out=self.output)
        np.multiply(self.output, 1./npos, where=~neg,
                    out=self.output)
        self.output = np.sum(self.output)
        return self.output

    def updateGradInput(self, input, target):
        # Your code goes here. ################################################
        return self.gradInput
    
    def __repr__(self):
        return "ClassContrastiveCriterion"

# Optimizers

### SGD optimizer with momentum
- `variables` - list of lists of variables (one list per layer)
- `gradients` - list of lists of current gradients (same structure as for `variables`, one array for each var)
- `config` - dict with optimization parameters (`learning_rate` and `momentum`)
- `state` - dict with optimizator state (used to save accumulated gradients)

In [126]:
def sgd_momentum(variables, gradients, config, state):  
    # 'variables' and 'gradients' have complex structure, accumulated_grads will be stored in a simpler one
    state.setdefault('accumulated_grads', {})
    
    var_index = 0 
    for current_layer_vars, current_layer_grads in zip(variables, gradients): 
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            
            old_grad = state['accumulated_grads'].setdefault(var_index, np.zeros_like(current_grad))
            
            np.add(config['momentum'] * old_grad, config['learning_rate'] * current_grad, out=old_grad)
            
            current_var -= old_grad
            var_index += 1     

## 12. [Adam](https://arxiv.org/pdf/1412.6980.pdf) optimizer
- `variables` - list of lists of variables (one list per layer)
- `gradients` - list of lists of current gradients (same structure as for `variables`, one array for each var)
- `config` - dict with optimization parameters (`learning_rate`, `beta1`, `beta2`, `epsilon`)
- `state` - dict with optimizator state (used to save 1st and 2nd moment for vars)

Formulas for optimizer:

Current step learning rate: $$\text{lr}_t = \text{learning_rate} * \frac{\sqrt{1-\beta_2^t}} {1-\beta_1^t}$$
First moment of var: $$\mu_t = \beta_1 * \mu_{t-1} + (1 - \beta_1)*g$$ 
Second moment of var: $$v_t = \beta_2 * v_{t-1} + (1 - \beta_2)*g*g$$
New values of var: $$\text{variable} = \text{variable} - \text{lr}_t * \frac{m_t}{\sqrt{v_t} + \epsilon}$$

In [127]:
def adam_optimizer(variables, gradients, config, state):  
    # 'variables' and 'gradients' have complex structure, accumulated_grads will be stored in a simpler one
    state.setdefault('m', {})  # first moment vars
    state.setdefault('v', {})  # second moment vars
    state.setdefault('t', 0)   # timestamp
    state['t'] += 1
    for k in ['learning_rate', 'beta1', 'beta2', 'epsilon']:
        assert k in config, config.keys()
    
    var_index = 0 
    lr_t = config['learning_rate'] * np.sqrt(1 - config['beta2']**state['t']) / (1 - config['beta1']**state['t'])
    for current_layer_vars, current_layer_grads in zip(variables, gradients): 
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            var_first_moment = state['m'].setdefault(var_index, np.zeros_like(current_grad))
            var_second_moment = state['v'].setdefault(var_index, np.zeros_like(current_grad))
            
            # <YOUR CODE> #######################################
            # update `current_var_first_moment`, `var_second_moment` and `current_var` values
            #np.add(... , out=var_first_moment)
            #np.add(... , out=var_second_moment)
            #current_var -= ...
            tmpg = (1. - config['beta1']) * current_grad
            np.multiply(var_first_moment, config['beta1'],
                        out=var_first_moment)
            np.add(var_first_moment, tmpg,
                   out=var_first_moment)
            np.multiply(1. - config['beta2'], current_grad,
                        out= tmpg)
            np.multiply(tmpg, current_grad,
                        out=tmpg)
            np.multiply(var_second_moment, config['beta2'],
                        out=var_second_moment)
            np.add(var_second_moment, tmpg,
                   out=var_second_moment)
            step = np.sqrt(var_second_moment)
            np.add(step, config['epsilon'], out=step)
            np.divide(var_first_moment, step, out=step)
            np.multiply(lr_t, step, out=step)
            np.subtract(current_var, step, out=current_var)
            
            # small checks that you've updated the state; use np.add for rewriting np.arrays values
            assert var_first_moment is state['m'].get(var_index)
            assert var_second_moment is state['v'].get(var_index)
            var_index += 1


### Flatten layer
Just reshapes inputs and gradients. It's usually used as proxy layer between Conv2d and Linear.

In [128]:
class Flatten(Module):
    def __init__(self):
         super(Flatten, self).__init__()
    
    def updateOutput(self, input):
        self.output = input.reshape(len(input), -1)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput.reshape(input.shape)
        return self.gradInput
    
    def __repr__(self):
        return "Flatten"